In [1]:

!pip install pyspark==3.1.2

In [2]:
from pyspark import SparkConf, SparkContext
from google.colab import files
import csv
conf = SparkConf().setAppName("grade_analysis")
sc = SparkContext(conf=conf)

In [3]:
uploaded = files.upload()

Saving grades(1).csv to grades(1) (1).csv


In [4]:
input_file_path = next(iter(uploaded))
data = sc.textFile(input_file_path).filter(lambda line: "First name" not in line)
raw_data = data.collect()
for line in raw_data:
    print(line)

1,Samarth,Urs,101,95
2,John,Smith,102,92
3,Michael,Brown,103,75
4,Samarth,Urs,104,91
5,Michael,Brown,105,93


In [5]:
#parsed_data = data.map(lambda line: line.split(",")).map(lambda fields: (fields[1].strip() + " " + fields[2].strip(), (float(fields[4]), 1)))
parsed_data = data.map(lambda line: line.split(",")).map(lambda fields: (fields[1] + " " + fields[2], (float(fields[4]), 1)))

min_max_grades = parsed_data.reduceByKey(lambda x, y: (min(x[0], y[0]), max(x[0], y[0])))
total_grades_count = parsed_data.reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))
gpa_courses = total_grades_count.mapValues(lambda x: (x[0] / x[1], x[1]))


In [6]:
result = gpa_courses.collect()

In [7]:
import os

# Set a fixed PYTHONHASHSEED value
os.environ["PYTHONHASHSEED"] = "0"

In [8]:
for student, (gpa, course_count) in result:
    print(f"Student: {student}")
    min_max_entry = min_max_grades.lookup(student)
    if min_max_entry:
        print(f"Minimum Grade: {min_max_entry[0][0]}")
        print(f"Maximum Grade: {min_max_entry[0][1]}")
    else:
        print("No grade data available for this student.")
    print(f"GPA: {gpa}")
    print(f"Number of Courses Taken: {course_count}")
    print("=" * 40)

# Stop the SparkContext
sc.stop()








Student: Samarth Urs
Minimum Grade: 91.0
Maximum Grade: 95.0
GPA: 93.0
Number of Courses Taken: 2
Student: John Smith
Minimum Grade: 92.0
Maximum Grade: 1
GPA: 92.0
Number of Courses Taken: 1
Student: Michael Brown
Minimum Grade: 75.0
Maximum Grade: 93.0
GPA: 84.0
Number of Courses Taken: 2
